In [6]:
import sqlite3
import pandas as pd

# List of database filenames
database_files = ["Treasury_Yields_data.db", "CoT_data.db", "FED_Data.db"]  # Add your database filenames

# Create an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Iterate through each database
for db_file in database_files:
    # Create a connection to the database
    conn = sqlite3.connect(db_file)

    # Get a list of all tables in the database
    tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = conn.execute(tables_query).fetchall()

    # Filter tables that contain the phrase "weekly_percent_change"
    relevant_tables = [table[0] for table in tables if "weekly_percent_change" in table[0]]

    # Iterate through relevant tables
    for table_name in relevant_tables:
        # Read the table into a DataFrame
        query = f"SELECT * FROM '{table_name}';"
        df = pd.read_sql_query(query, conn, index_col='Date', parse_dates='Date')

        # Extract necessary columns (you may adjust this based on your data structure)
        #relevant_columns = ["column1", "column2", ...]  # Replace with your relevant column names
        #df = df[relevant_columns]
        # Rename columns by combining table name and existing column name
        new_columns = [f"{table_name}_{col}" for col in df.columns]
        df.columns = new_columns
        
        # Merge or join with the existing merged_data
        merged_data = pd.merge(merged_data, df, how='outer', left_index=True, right_index=True)

    # Close the database connection
    conn.close()

# If you want to fill NaN values with a specific value, you can use:
# merged_data = merged_data.fillna(your_value)

# Print or use the merged_data DataFrame
#print(merged_data)


In [9]:
import numpy as np

df2 = merged_data.copy()

# Define the features (independent variables) and target (dependent variable)
features = list(df2)

# Define the lag order (number of lags to create)
lag_order = 6  # Example: Creating lagged values for the past n periods

for feature in features:
    for lag in range(1, lag_order+1):
        df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)


# replace inf and -inf with na and  drops rows with missing values in the lagged variables
#df2 = df2.replace([np.inf, -np.inf], np.nan).dropna()


C:\Users\dstoc\AppData\Local\Temp\ipykernel_30244\646358871.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)
C:\Users\dstoc\AppData\Local\Temp\ipykernel_30244\646358871.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)
C:\Users\dstoc\AppData\Local\Temp\ipykernel_30244\646358871.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor pe

In [10]:
import pickle
AllVar_weekly = df2.copy()
# Serialize the object using pickle and save to a file
with open('AllVar_weekly.pkl', 'wb') as file:
    pickle.dump(AllVar_weekly, file)



In [4]:
AllVar_weekly.tail()

,YIELD_weekly_percent_change_1 MO,YIELD_weekly_percent_change_2 MO,YIELD_weekly_percent_change_3 MO,YIELD_weekly_percent_change_6 MO,YIELD_weekly_percent_change_1 YR,YIELD_weekly_percent_change_2 YR,YIELD_weekly_percent_change_3 YR,YIELD_weekly_percent_change_5 YR,YIELD_weekly_percent_change_7 YR,YIELD_weekly_percent_change_10 YR,...,SOMA_weekly_percent_change_agencies_lag1,SOMA_weekly_percent_change_agencies_lag2,SOMA_weekly_percent_change_agencies_lag3,SOMA_weekly_percent_change_agencies_lag4,SOMA_weekly_percent_change_agencies_lag5,SOMA_weekly_percent_change_total_lag1,SOMA_weekly_percent_change_total_lag2,SOMA_weekly_percent_change_total_lag3,SOMA_weekly_percent_change_total_lag4,SOMA_weekly_percent_change_total_lag5
Date,,,,,,,,,,,,,,,,,,,,,
2023-10-29,0.001799,0.001799,0.001792,0.001805,-0.003697,-0.015779,-0.018256,-0.020576,-0.020284,-0.018256,...,0.0,0.0,0.0,0.0,0.0,-0.002136,-0.000325,-0.003832,-0.002388,-0.003576
2023-11-05,-0.007181,-0.001795,-0.010733,-0.018018,-0.018553,-0.032064,-0.045455,-0.056723,-0.057971,-0.055785,...,0.0,0.0,0.0,0.0,0.0,-0.002536,-0.002136,-0.000325,-0.003832,-0.002388
2023-11-12,0.000000,-0.001799,0.000000,0.001835,0.017013,0.043478,0.038961,0.035635,0.028571,0.008753,...,0.0,0.0,0.0,0.0,0.0,-0.005511,-0.002536,-0.002136,-0.000325,-0.003832
2023-11-19,-0.001808,-0.001802,-0.005425,-0.012821,-0.026022,-0.031746,-0.033333,-0.043011,-0.044872,-0.036876,...,0.0,0.0,0.0,0.0,0.0,0.000003,-0.005511,-0.002536,-0.002136,-0.000325
2023-11-26,0.005435,0.003610,0.007273,0.011132,0.005725,0.008197,0.006466,0.008989,0.008949,0.006757,...,0.0,0.0,0.0,0.0,0.0,-0.004228,0.000003,-0.005511,-0.002536,-0.002136


In [5]:

# Define the number of periods to lead the target variable (1 period in this case)
lead_periods = 1

# Create a new variable representing the 'lead' of the target
df2['YIELD_weekly_percent_change_10 YR_lead'] = df2['YIELD_weekly_percent_change_10 YR'].shift(-lead_periods)

# Drop rows with missing values in the features or target (created by the shift)
df2 = df2.dropna(subset=features + ['YIELD_weekly_percent_change_10 YR_lead'])

C:\Users\dstoc\AppData\Local\Temp\ipykernel_30244\512717327.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2['YIELD_weekly_percent_change_10 YR_lead'] = df2['YIELD_weekly_percent_change_10 YR'].shift(-lead_periods)
